# Semantic Program Analysis for Scientific Model Augmentation

![SemanticModels Logo](src/img/semanticmodels_jl.dot.svg)

- Teaching computers to do science
- Model Augmentation and Synthesis
- Arbitrary models are complex, but transformations are simpler
- Project Repo [github.com/jpfairbanks/SemanticModels.jl](github.com/jpfairbanks/SemanticModels.jl)

## Machine Augmentation of Scientists

- Scientists are busy writting papers
- Papers are a low bandwidth medium, because of imprecision
- We want to build AI scientists


## Science as nested optimization

Fitting the data is a regression problem:

$$\ell^* = \min_{h\in {H}} \ell(h(x), y)$$ 

Institutional process of discovery is

$$\max_{{H}\in \mathcal{M}} expl(h^*)$$ where *expl* is the explanatory power of a class of models $H$. 

- The explanatory power is some combination of 
    - generalization,
    - parsimony,
    - and consistency with the fundamental principles of the field.

## Modeling Frameworks

Most frameworks are designed before the models are written

| Framework | Math | Input Specification  | Description |
|-----------|------|----------|-------------|
| <img width="50px" src="https://www.mathworks.com/content/dam/mathworks/mathworks-dot-com/images/ico_membrane_128.png"> Matlab/Scipy</img> | x = A\b | BLAS + scripting | Sci/Eng math is all BLAS| 
| <img width="50px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Mathematica_Logo.svg/1920px-Mathematica_Logo.svg.png"> Mathematica</img> |$p(x)=0$|Symbolic Math Expressions| Computer Algebra Systems|
| <img width="50px" src="https://mc-stan.org/docs/2_18/stan-users-guide/img/logo-tm.png" alt="Stan Logo">Stan</img> | $ y \sim \mathcal{N}(x \beta + \alpha, \sigma)$ | StanML | Bayesian Inference|
| <img src="https://camo.githubusercontent.com/31d60f762b44d0c3ea47cc16b785e042104a6e03/68747470733a2f2f7777772e6a756c69616f70742e6f72672f696d616765732f6a756d702d6c6f676f2d776974682d746578742e737667" alt="jump"></img> | $\min_{x\perp C(x)} f(x)$ | AMPL based DSL| Optimization Problems |
| ![TensorFlow](https://www.gstatic.com/devrel-devsite/va3a0eb1ff00a004a87e2f93101f27917d794beecfd23556fc6d8627bba2ff3cf/tensorflow/images/lockup.svg) | $y\approx f(x)$ | TF.Graph| Deep Learning|
| <img width="75px" src="http://aske.gtri.gatech.edu/docs/latest/img/semanticmodels_jl.dot.svg">SemanticModels.jl</img> | All Computable Domains| Julia Programs | $Models \subset Code$|

SemanticModels is a post hoc modeling framework

## Modeling Frameworks

Most frameworks are designed before the models are written

|Domain | <p></p> | <p></p> |
|--|-----------|------|
|Algebra | <img width="50px" src="https://www.mathworks.com/content/dam/mathworks/mathworks-dot-com/images/ico_membrane_128.png" alt="Matlab/Scipy"></img>  | <img width="50px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Mathematica_Logo.svg/1920px-Mathematica_Logo.svg.png" alt="Mathematica"></img> |
|Learning | <img width="50px" src="https://mc-stan.org/docs/2_18/stan-users-guide/img/logo-tm.png" alt="Stan"></img> |  ![TensorFlow](https://www.gstatic.com/devrel-devsite/va3a0eb1ff00a004a87e2f93101f27917d794beecfd23556fc6d8627bba2ff3cf/tensorflow/images/lockup.svg) |
|Optimization | <img width="75px" src="https://camo.githubusercontent.com/31d60f762b44d0c3ea47cc16b785e042104a6e03/68747470733a2f2f7777772e6a756c69616f70742e6f72672f696d616765732f6a756d702d6c6f676f2d776974682d746578742e737667" alt="jump"></img> | <img width="50px" src="https://pbs.twimg.com/media/C7dmXwGXQAA47XE.jpg:large" alt="CPLEX"></img> |
|Modeling | <img width="50px" src="http://docs.juliadiffeq.org/latest/assets/logo.png" alt="JuliaDiffeq"></img> | <img width="75px" src="http://aske.gtri.gatech.edu/docs/latest/img/semanticmodels_jl.dot.svg"></img> |

SemanticModels is a post hoc modeling framework

## Statistical / ML models are accurate

Fitting curves to data is good, but doesn't explain the data.

## Scientific Models are Mechanistic
Mechanistic models are more explainable than black box or statistical models. They posit driving forces and natural laws
that drive the evolution of systems over time.

We call these *simulations* when necessary to distinguish from *model*

- Benefits: more explainable, more generalizable
- Cons: lower Accuracy, less flexible

## SIR model of disease

![SIR](src/img/cd/tikzit/sir_petri+ode.png)

### ODE based simulation

#### A mathematical model of disease spread

\begin{align}
\frac{dS}{dt}&=-\frac{\beta IS}{N}\\\\
\frac{dI}{dt}&=\frac{\beta IS}{N}-\gamma I\\\\
\frac{dR}{dt}&=\gamma I
\end{align}

#### Predictions

![Ebola Outbreak](src/img/srep42594-f1-cropped.png)
- (a) Cumulative number of infected individuals as a function of time (day) for the three countries Guinea, Liberia and Sierra Leone. 
- A Khalequea, and P Senb, "An empirical analysis of the Ebola outbreak in West Africa" 2017

![SIR solution](https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Sirsys-p9.png/330px-Sirsys-p9.png)

## ODE Based Implementation

This is a "real world" implementation of SIR modeling in Julia taken from Epirecipes Cookbook (Simon Frost)

```julia
module SIRModel
using DifferentialEquations

function sir_ode(du, u, p, t)
    #Infected per-Capita Rate
    β = p[1]
    #Recover per-capita rate
    γ = p[2]
    #Susceptible Individuals
    S = u[1]
    #Infected Individuals
    I = u[2]
    du[1] = -β * S * I
    du[2] = β * S * I - γ * I
    du[3] = γ * I
end

#Param = (Infected Per Capita Rate, Recover Per Capita Rate)
param = [0.1,0.05]
#Initial Params = (Susceptible Individuals, Infected by Infected Individuals)
init = [0.99,0.01,0.0]
tspan = (0.0,200.0)

sir_prob = ODEProblem(sir_ode, init, tspan, param)

sir_sol = solve(sir_prob, saveat = 0.1);
```

## Julia Programs as a Category

1. $Ob(C) = Types$
2. $Hom_C(S,T) = S\rightarrow T$ functions

![Julia Types SIR](src/img/types_sir.dot.png)

### Agent based simulation


```julia
""" Agent Models is a hypothetical ABM framework"""
module AgentModels

abstract type AgentModel end
mutable struct StateModel <: AgentModel
    states
    agents
    transitions
end

solve(m::StateModel) = return thesolution(m)
end
```

```julia
using AgentModels #<- hypothetical ABM framework 

numinfected() = sum(a.==:I)
infection(x) = rand(Float64) < numinfected() ? :I : :S
recovery(x) = rand(Float64) < ρ ? :I : :R

function main(nsteps)
    n = 20
    a = fill(:S, n)
    ρ = 0.5 + randn(Float64)/4 # chance of recovery
    μ = 0.5 # chance of immunity
    T = Dict(S=>infection, I=>recovery)
    sam = StateModel([:S, :I, :R], a, T, zeros(Float64,3))
    distribution = solve!(sam, nsteps)
    return distribution
end

main(150)
```

## Model Augmentation as a Lens

We want scientists to program using lenses

![Module Augmentation as a Lens](src/img/cd/model_lens.png)

*What should the $M_1, M_2$ be*?

## Objects from a Category!
![Petri Net for SIR](src/img/cd/tikzit/sir.png)

## Petrinet SIR Model

```julia
using Petri

function main()
    @variables S, I, R
    N = +(S,I,R)

    Δ = [(S+I, 2I),
         (I, R)]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRState(100, 1, 0), 50)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = main()

```

## Statistical Models

Linear Regression is a model.
```julia
using LsqFit
function f(x, β) 
    return β[1] .* x + β[2]
end

function main()
    X = load_matrix("file_X.csv")
    target = load_vector("file_y.csv")
    a₀ = [1.0, 1.0]
    
    fit = curve_fit(f, X, target, a₀)
    return fit
end

main()
```

## Category Theory

CT is the mathematics of structure preserving maps. Every field of math has a notion of *homomorphism* where two objects
in that category *have similar structure*

1. Sets, Groups, Fields, Rings
2. Graphs
3. Databases

CT is the study of structure in its most general form.

## Categories as Objects and Morphisms

1. Categories have Objects, $ Ob(C) $
2. Morphisms are relations between objects $Hom_C(x,y)$
3. There is always an identity morphism $id_x \in Hom_C(x,y)$
4. Morphisms Compose $f\in Hom_C(x,y), g\in Hom_C(y,z)$ then $g\circ f \in Hom_C(x,z)$ 


## Graphs as Categories

### Each graph is a category
- $ G = (V,E) $
- $Ob(G) = V$
- $Hom_G(v,u) = \{(v\leadsto u) \in E\}$

<!--\{e \mid e\in E, src(e)=v \land dst(e)=u\}$ -->

### The category of graphs

- Graph Homomorphism $f: G\to H$ st $(v\leadsto u) \in G \implies (f(v) \leadsto f(u)) \in H$
- $Ob(Graph)$ is the set of all graphs
- $Hom_{Graph}(G,H)$ is the set of all graph homomorphisms $G$ to $H$



## Semantic Models Applies Category Theory

A novel modeling environment that builds and manipulates models in this category theory approach.

Contributions: 
1. We take general code as input
2. Highly general and extensible framework
3. Goal: Transformations are compositional

### The category of models

While each model can be represented as a category, there is a category of all models.

Functors between models are the morphisms in this category.

<img src="src/img/cd/modelcat.png"></img>

<!-- <img src="src/img/cd/gravity.svg"></img> -->

# Structural Model Changes
Modifying models using a Grammar of rewrite rules.

Double Push Outs over $\mathcal{Petri}$ (Cicala 2019 SYCO 4) 

## Petrinet Model

```julia
using Petri

function main()
    @variables S, I, R
    N = +(S,I,R)

    Δ = [(S+I, 2I),
        (I, R),]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRState(100, 1, 0), 50)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = main()
```


## SIR -> SIRS
![SIR model](src/img/cd/tikzit/rule_sir_sirs.png)

## SIRS model as code

```julia
using Petri

function main()
    @variables S, I, R
    N = +(S,I,R)

    Δ = [(S+I, 2I),
        (I, R),
        (R, S)]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRState(100, 1, 0), 50)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = main()
```


![An more refined ABM](src/img/cd/tikzit/sir_dpo.png)

## Software Interface for Rewriting Models
```julia
states = [S, I, R]

sir = Petri.Model(states,[(I, R), (S+I, 2I)])
ir = Petri.Model(states, [(I, R)])
seir = Petri.Model(states, [(I, R), (S+I, I+E), (E, I)])
rule = Span(sir, ir, seir)

# the root of the bottom of DPO
irs = Petri.Model(states, [(I, R), (R, S)])

sirs, seirs = solve(DPOProblem(rule, irs))
```


## SEIRS Model as Declarative Code

```julia
using Petri

function SEIRSmain()
    @variables S, E, I, R
    N = +(S,E,I,R)

    Δ = [(S+I, I+E),
         (E, I),
         (I, R),
         (R, S) 
    ]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SEIRState(100, 0, 1, 0), 150)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = SEIRSmain()

```

## SEIRS Model as Imperative Code
```julia
 :(##δ#754(state) = begin
          begin
              begin
                  state.I > 0 || return nothing
                  state.I -= 1
              end
              state.R += 1
          end
      end)                                                                                                                                                        
 :(##δ#755(state) = begin
          begin
              begin
                  state.S > 0 || return nothing
                  state.I > 0 || return nothing
                  state.S -= 1
                  state.I -= 1
              end
              begin
                  state.I += 1
                  state.E += 1
              end
          end
      end)
 :(##δ#756(state) = begin
          begin
              begin
                  state.E > 0 || return nothing
                  state.E -= 1
              end
              state.I += 1
          end
      end)                                                                                                                                                        
 :(##δ#757(state) = begin
          begin
              begin
                  state.R > 0 || return nothing
                  state.R -= 1
              end
              state.S += 1
          end
end)                            
```

## Compiler / Disassembler

![Module Augmentation as a Lens](src/img/cd/tikzit/model_lens_2.png)



## SEIRS Model as x86 Assembly
A tiny portion of the model!

```
	.section	__TEXT,__text,regular,pure_instructions
; Function ##δ#788 {
; Location: rewrite.jl:127
; Function getproperty; {
; Location: rewrite.jl:148
	decl	%eax
	movl	8(%esi), %eax
;}
; Function >; {
; Location: operators.jl:286
; Function <; {
; Location: int.jl:49
	decl	%eax
	testl	%eax, %eax
;}}
	jle	L37
; Function -; {
; Location: int.jl:52
	decl	%eax
	addl	$-1, %eax
;}
; Function setproperty!; {
; Location: sysimg.jl:19
	decl	%eax
	movl	%eax, 8(%esi)
;}
; Function getproperty; {
; Location: sysimg.jl:18
	decl	%eax
	movl	16(%esi), %eax
;}
; Function +; {
; Location: int.jl:53
	decl	%eax
	addl	$1, %eax
;}
; Function setproperty!; {
; Location: sysimg.jl:19
	decl	%eax
	movl	%eax, 16(%esi)
;}
	decl	%eax
	movl	%eax, (%edi)
	movb	$2, %dl
	xorl	%eax, %eax
	retl
L37:
	movb	$1, %dl
	xorl	%eax, %eax
; Location: rewrite.jl:127
	retl
	nopw	(%eax,%eax)
;}
```

## Compiling Petri.Model to ODEProblem
```julia
f(du, state, p, t) = begin
    du.S = (-p[1] * (((state.β * state.S) * state.I) 
                      / ((state.S + state.I) + state.R))) 
            + p[3] * (state.μ * state.R)
    
    du.I = (p[1] * (((state.β * state.S) * state.I) 
                      / ((state.S + state.I) + state.R))) 
            + -p[2] * (state.γ * state.I)
    
    du.R = ((p[2] * (state.γ * state.I)) 
            + -p[3] * (state.μ * state.R))
end

function main()
  param = [0.1,0.05]
  init = [0.99,0.01,0.0]
  tspan = (0.0,200.0)

  prob = ODEProblem(f, init, tspan, param)

  sol = solve(prob);
end

```

## Model Changes Require Coordinated Changes Across Code
Adding un estado de lo muertos
<img src="https://docs.google.com/drawings/d/e/2PACX-1vRUhrX6GzMzNRWr0GI3pDp9DvSqJVTDVpy9SNNBIB08b7Hyf9vaHobE2knrGPda4My9f_o9gncG34pF/pub?w=1028&amp;h=309">

If I add a state (place), I have to make coordinated changes. 

## SIRD Model

```julia
using Petri

function SIRDmain()
    @variables S, I, R, D
    N = +(S,E,I,R, D)

    Δ = [(S+I, 2I),
         (I, R),
         (R, S),
         (I, D)
    ]

    m = Petri.Model(Δ)
    p = Petri.Problem(m, SIRDState(100, 1, 0, 0), 150)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = SIRDmain()
```

## Structural Model Changes
Adding Population growth
<img src="https://docs.google.com/drawings/d/e/2PACX-1vRfLcbPPaQq6jmxheWApqidYte8FxK7p0Ebs2EyW2pY3ougNh5YiMjA0NbRMuGAIT5pD02WNEoOfdCd/pub?w=1005&amp;h=247">

## SIRD Model with Birth Process

```julia
using Petri

function BSEIRDmain()
    @variables S, E, I, R, β, γ, μ, D, ψ
    N = +(S,E,I,R)

    Δ = [(S~S-1, I~I+1),
         (I~I-1, R~R+1),
         (R~R-1, S~S+1),
         (I~I-1, D~D+1),
         (S~S+1,) # birth process
     ]

    Λ = [β*S*I/N,
         γ*I,
         μ*R,
         ψ*I,
         ν*S # birth rate
    ]

    m = Petri.Model(Δ, Λ)
    p = Petri.Problem(m, BSIRDState(100, 1, 0, 0.5, 0.15, 0.05, 0, 0.1, 0.1), 150)
    soln = Petri.solve(p)
    (p, soln)
end
p, soln = BSEIRDmain()

```

## Model Augmentation yields polynomial regression

### Goal

Linear regression: $\min_\beta \| y - \beta x\|$

Polynomial regression: $\min_\beta \| y - f_\beta(x)\|$

### Augmentations
Given transformations

- $T_1 = f(x) \mapsto f(x) + 1$
- $T_x = f(x) \mapsto x\cdot f(x)$
- $T_1 \circ T_1 = T_1$ (idempotent)

we are able to generate all possible polynomial regression using composition of these transformations.

## Model Selection
<img src="src/img/cd/poly.png"></img>

## Converting Models between Categories
Models can be represented in different categories, for example, SIR as an OLOG.
![An SIR model structure](src/img/olog_sir.dot.svg)

### Category of Models

![The family of compartmental models](src/img/compartmental.dot.svg)

## Type Graphs
1. The TypeGraph of a Julia Program looks a lot like the OLOG of the program it runs
2. Computers are good at type checking
3. Can we embed our semantics into the type system?


![Julia Types SIR](src/img/types_sir.dot.png)


![Resolving Type Ambiguities](src/img/type_ambig_resolve.png)

# Functorial Semantics?

Can we make this rigorous?
![Functorial Semantics?](src/img/cd/tikzit/model_driven.png)

## Types for our ABM example
![An ABM of SIR disease spread](src/img/exampletypegraph.dot.svg)

## Refining the model 

Convert categorical values into singleton types:

![An more refined ABM](src/img/typegraphmorphism.dot.svg)

## The type system "understands" the agents now

Convert categorical values into singleton types:

![An more refined ABM](src/img/type_DFA.dot.svg)

# Conclusion

1. SemanticModels.jl github.com/jpfairbanks/SemanticModels.jl is a foundational technology for teaching machines to reason about scientific models

2. SemanticModels.jl combines DPO rewriting with Lenses for model augmentation **for science!**

3. $SemanticModels = Codification \circ Categorification \circ Science $

## Open Questions

1. Which scientific modeling frameworks can be categorified? 
2. How can we compute DPO rewriting for general categories?
3. What other modeling activities have CT formalizations?


# Acknowledgements
![Acknowledgements](src/img/Acknowledgements_ASKE.png)